<a href="https://colab.research.google.com/github/Jisang-hwang93/BOK_Report_Team1/blob/master/report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import csv
import pandas as pd
import datetime

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# tika
- 채권분석 레포트 token화, ngram화 한 데이터

In [37]:
tika = pd.read_csv('/content/drive/My Drive/자연어 처리 1조/데이터셋/유연/toknized_bond_report.csv', header=[0], encoding='utf-8')
# tika['new_date'] = tika.날짜.apply(lambda x: datetime.datetime.strptime(x, "%y.%m.%d"))
# tika.drop('날짜', axis=1, inplace=True)
# tika['before_month'] = tika.new_date.apply(lambda x: x - datetime.timedelta(days = 30))
tika

0  ... 기대/NNG;글로벌/NNG;금리/NNG;상승/NNG,하락/NNG;예상/NNG;금리/NNG;인상/NNG,발행/NNG;늘/VV;금리/NNG;상승/NNG,금리/NNG;인상/NNG;확신/NNG;없/VA,장기물/NNG;발행비중/NNG;확대/NNG,국고채/NNG;금리/NNG;상승/NNG,장기물/NNG;수요/NNG;증가/NNG,부정적/VAX;영향/NNG;상쇄/NNG,금리/NNG;하락/NNG;상승/NNG,국제/NNG;유가/NNG;상승/NNG,물가/NNG;상승/NNG;압력/NNG,금리/NNG;상승/NNG;아니/VCN,금리/NNG;상승/NNG;압력/NNG,아직/MAG;경기/NNG;회복/NNG,고용/NNG;개선/NNG;미흡/NNG,내수/NNG;경기/NNG;부양/NNG,소비/NNG;지표/NNG;개선/NNG,금리/NNG;인상/NNG;어렵/VA,물가/NNG;상승률/NNG;넘/VV,국채/NNG;발행/NNG;늘/VV,농축수산물/NNG;하락/NNG,수요우위/NNG;해소/NNG,위험자산/NNG;선호/NNG,금리/NNG;부정적/VAX,부양/NNG;효과/NNG,금리/NNG;반대/NNG,경제/NNG;완화/NNG,회복/NNG;조짐/NNG,경기/NNG;과열/NNG,하락/NNG;효과/NNG,공급/NNG;확대/NNG,채권/NNG;증가/NNG,금리/NNG;오르/VV,어렵/VA;위험/NNG,금리/NNG;어렵/VA,폐쇄/NNG;막/VV
0        1  ...  국고채/NNG;금리/NNG;급등/NNG,원금/NNG;상환/NNG;부담/NNG,금리/...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
1        2  ...  금리/NNG;인상/NNG;반대/NNG,희박/NNG;금리/NNG;인상/NNG,아직/M...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
2        3  ...  크레딧/NNG;시장/NNG;스프레드/NNG;축소/NNG,전망/NNG;상향/NNG;전...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
3        4  ...  fed/NNG;ff/NNG;목표/NNG;금리/NNG;인상/NNG,금리/NNG;인상/...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
4        5  ...  fed/NNG;시장/NNG;예상/NNG;금리/NNG;인상/NNG,실업률/NNG;하락...                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

# ir (interest rate)
- 날짜에 따른 콜금리 & 기준금리 데이터
- 콜금리 기준이므로 기준금리 컬럼은 삭제

In [10]:
ir = pd.read_csv('/content/drive/My Drive/자연어 처리 1조/데이터셋/interest_rate.csv', encoding='utf-8')
ir.rename(columns = {'Unnamed: 0' : 'date'}, inplace = True)
ir['new_date'] = ir.date.apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
ir.drop(['date', '기준 금리'], axis=1, inplace=True)
ir['before_month'] = ir['new_date']
ir.head()

,콜 금리,new_date,before_month
0,NaN,1999-05-06,1999-05-06
1,NaN,2000-02-10,2000-02-10
2,NaN,2000-10-05,2000-10-05
3,NaN,2001-02-08,2001-02-08
4,NaN,2001-07-05,2001-07-05
...,...,...,...
3885,0.44,2020-07-20,2020-07-20
3886,0.46,2020-07-21,2020-07-21
3887,0.45,2020-07-22,2020-07-22
3888,0.46,2020-07-23,2020-07-23


# ngram과 콜금리 변화에 따른 라벨링


In [24]:
data = pd.merge(tika, ir, on='new_date')
data.drop('Unnamed: 0', axis=1, inplace = True)
data.rename(columns = {'before_month_x' : 'before_month'}, inplace = True)
data = pd.merge(data, ir, on='before_month')
data['change'] = data['콜 금리_x'] - data['콜 금리_y']
data

,제목,내용,new_date_x,before_month,콜 금리_x,before_month_y,콜 금리_y,new_date_y,change
0,트럼프노믹스를 극복한 2017년 시장,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,2017-12-27,2017-11-27,1.51,2017-12-27,1.26,2017-11-27,0.25
1,2금융권 안심전환대출 수급영향 점검,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,2017-12-20,2017-11-20,1.51,2017-12-20,1.25,2017-11-20,0.26
2,확신 없는 인상의견도 있었음이 확인,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,2017-12-20,2017-11-20,1.51,2017-12-20,1.25,2017-11-20,0.26
3,성장과 물가의 Dilemma,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,2017-12-15,2017-11-15,1.51,2017-12-15,1.24,2017-11-15,0.27
4,12월 FOMC 함의와 2018년 연준구성 변화,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,2017-12-15,2017-11-15,1.51,2017-12-15,1.24,2017-11-15,0.27
...,...,...,...,...,...,...,...,...,...
1318,대우증권 5월 29일 채권 시황,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,2008-05-29,2008-04-29,5.03,2008-05-29,4.94,2008-04-29,0.09
1319,대우증권 5월 28일 채권 시황,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,2008-05-28,2008-04-28,5.03,2008-05-28,4.89,2008-04-28,0.14
1320,대우증권 5월 23일 채권 시황,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,2008-05-23,2008-04-23,5.00,2008-05-23,4.98,2008-04-23,0.02
1321,대우증권 5월 22일 채권 현황,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,2008-05-22,2008-04-22,5.00,2008-05-22,4.98,2008-04-22,0.02


In [13]:
# 콜금리 변화에 따른 라벨링

def labeling(change):
    if change > 0.03:
        return 1
    elif change < -0.03:
        return -1
    else:
        return 0

In [25]:
data['label'] = data.change.apply(lambda x: labeling(x))
data

,제목,내용,new_date_x,before_month,콜 금리_x,before_month_y,콜 금리_y,new_date_y,change,label
0,트럼프노믹스를 극복한 2017년 시장,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,2017-12-27,2017-11-27,1.51,2017-12-27,1.26,2017-11-27,0.25,1
1,2금융권 안심전환대출 수급영향 점검,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,2017-12-20,2017-11-20,1.51,2017-12-20,1.25,2017-11-20,0.26,1
2,확신 없는 인상의견도 있었음이 확인,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,2017-12-20,2017-11-20,1.51,2017-12-20,1.25,2017-11-20,0.26,1
3,성장과 물가의 Dilemma,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,2017-12-15,2017-11-15,1.51,2017-12-15,1.24,2017-11-15,0.27,1
4,12월 FOMC 함의와 2018년 연준구성 변화,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,2017-12-15,2017-11-15,1.51,2017-12-15,1.24,2017-11-15,0.27,1
...,...,...,...,...,...,...,...,...,...,...
1318,대우증권 5월 29일 채권 시황,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,2008-05-29,2008-04-29,5.03,2008-05-29,4.94,2008-04-29,0.09,1
1319,대우증권 5월 28일 채권 시황,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,2008-05-28,2008-04-28,5.03,2008-05-28,4.89,2008-04-28,0.14,1
1320,대우증권 5월 23일 채권 시황,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,2008-05-23,2008-04-23,5.00,2008-05-23,4.98,2008-04-23,0.02,0
1321,대우증권 5월 22일 채권 현황,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,2008-05-22,2008-04-22,5.00,2008-05-22,4.98,2008-04-22,0.02,0


In [26]:
data['token_ngram'] = data[1] + data[2]
data

KeyError: ignored

In [141]:
data_original = data[['new_date_x', 'before_month', 1, 2, 'token_ngram', '콜 금리_x', '콜 금리_y', 'label']]
data_original

,new_date_x,before_month,1,2,token_ngram,콜 금리_x,콜 금리_y,label
0,2017-12-27,2017-11-27,"프레젠테이션/NNG,채권/NNG,채권시장/NNG,전망/NNG,동향/NNG,금리/NN...","기대/NNG;글로벌/NNG;금리/NNG;상승/NNG,하락/NNG;예상/NNG;금리/...","프레젠테이션/NNG,채권/NNG,채권시장/NNG,전망/NNG,동향/NNG,금리/NN...",1.51,1.26,1
1,2017-12-20,2017-11-20,"의사록/NNG,확신/NNG,없/VA,인상/NNG,의견/NNG,있/VA,확인/NNG,...","금리/NNG;인상/NNG;반대/NNG,희박/NNG;금리/NNG;인상/NNG,아직/M...","의사록/NNG,확신/NNG,없/VA,인상/NNG,의견/NNG,있/VA,확인/NNG,...",1.51,1.25,1
2,2017-12-20,2017-11-20,"속도조절/NNG,중/NNG,발행시장/NNG,공모/NNG,회사채/NNG,발행/NNG,...","크레딧/NNG;시장/NNG;스프레드/NNG;축소/NNG,전망/NNG;상향/NNG;전...","속도조절/NNG,중/NNG,발행시장/NNG,공모/NNG,회사채/NNG,발행/NNG,...",1.51,1.25,1
3,2017-12-15,2017-11-15,"fed/NNG,fed/NNG,함의/NNG,fed/NNG,구성/NNG,변화/NNG,채...","fed/NNG;시장/NNG;예상/NNG;금리/NNG;인상/NNG,실업률/NNG;하락...","fed/NNG,fed/NNG,함의/NNG,fed/NNG,구성/NNG,변화/NNG,채...",1.51,1.24,1
4,2017-12-15,2017-11-15,"채권분석/NNG,채권/NNG,fed/NNG,통화정책/NNG,불확실성/NNG,확대/N...","fed/NNG;금리/NNG;인상/NNG;가능성/NNG;작/VA,금리/NNG;인상/N...","채권분석/NNG,채권/NNG,fed/NNG,통화정책/NNG,불확실성/NNG,확대/N...",1.51,1.24,1
...,...,...,...,...,...,...,...,...
1318,2008-05-29,2008-04-29,"채권/NNG,시황/NNG,만기/NNG,금리/NNG,하락/NNG,하/VV,만기/NNG...","소비자/NNG;신뢰지수/NNG;예상/NNG;하회/NNG,물가/NNG;상승/NNG;압...","채권/NNG,시황/NNG,만기/NNG,금리/NNG,하락/NNG,하/VV,만기/NNG...",5.03,4.94,1
1319,2008-05-28,2008-04-28,"채권/NNG,시황/NNG,만기/NNG,금리/NNG,상승/NNG,하/VV,만기/NNG...","미/NNG;국채/NNG;수익률/NNG;급락/NNG,낙찰/NNG;금리/NNG;예상/N...","채권/NNG,시황/NNG,만기/NNG,금리/NNG,상승/NNG,하/VV,만기/NNG...",5.03,4.89,1
1320,2008-05-23,2008-04-23,"채권/NNG,시황/NNG,만기/NNG,금리/NNG,상승/NNG,하/VV,만기/NNG...","금리/NNG;하락/NNG;가능성/NNG;낮/VA,당분간/MAG;금리/NNG;하락/N...","채권/NNG,시황/NNG,만기/NNG,금리/NNG,상승/NNG,하/VV,만기/NNG...",5.00,4.98,0
1321,2008-05-22,2008-04-22,"채권/NNG,시황/NNG,만기/NNG,금리/NNG,하락/NNG,하/VV,만기/NNG...","물가/NNG;상승/NNG;압력/NNG,물가/NNG;상승/NNG;견인/NNG,금리/N...","채권/NNG,시황/NNG,만기/NNG,금리/NNG,하락/NNG,하/VV,만기/NNG...",5.00,4.98,0


In [143]:
data_original.columns = ['new_date', 'before_month', 'token', 'ngram', 'token_ngram', 'call_rate_now', 'call_rate_before', 'label']
data_original

,new_date,before_month,token,ngram,token_ngram,call_rate_now,call_rate_before,label
0,2017-12-27,2017-11-27,"프레젠테이션/NNG,채권/NNG,채권시장/NNG,전망/NNG,동향/NNG,금리/NN...","기대/NNG;글로벌/NNG;금리/NNG;상승/NNG,하락/NNG;예상/NNG;금리/...","프레젠테이션/NNG,채권/NNG,채권시장/NNG,전망/NNG,동향/NNG,금리/NN...",1.51,1.26,1
1,2017-12-20,2017-11-20,"의사록/NNG,확신/NNG,없/VA,인상/NNG,의견/NNG,있/VA,확인/NNG,...","금리/NNG;인상/NNG;반대/NNG,희박/NNG;금리/NNG;인상/NNG,아직/M...","의사록/NNG,확신/NNG,없/VA,인상/NNG,의견/NNG,있/VA,확인/NNG,...",1.51,1.25,1
2,2017-12-20,2017-11-20,"속도조절/NNG,중/NNG,발행시장/NNG,공모/NNG,회사채/NNG,발행/NNG,...","크레딧/NNG;시장/NNG;스프레드/NNG;축소/NNG,전망/NNG;상향/NNG;전...","속도조절/NNG,중/NNG,발행시장/NNG,공모/NNG,회사채/NNG,발행/NNG,...",1.51,1.25,1
3,2017-12-15,2017-11-15,"fed/NNG,fed/NNG,함의/NNG,fed/NNG,구성/NNG,변화/NNG,채...","fed/NNG;시장/NNG;예상/NNG;금리/NNG;인상/NNG,실업률/NNG;하락...","fed/NNG,fed/NNG,함의/NNG,fed/NNG,구성/NNG,변화/NNG,채...",1.51,1.24,1
4,2017-12-15,2017-11-15,"채권분석/NNG,채권/NNG,fed/NNG,통화정책/NNG,불확실성/NNG,확대/N...","fed/NNG;금리/NNG;인상/NNG;가능성/NNG;작/VA,금리/NNG;인상/N...","채권분석/NNG,채권/NNG,fed/NNG,통화정책/NNG,불확실성/NNG,확대/N...",1.51,1.24,1
...,...,...,...,...,...,...,...,...
1318,2008-05-29,2008-04-29,"채권/NNG,시황/NNG,만기/NNG,금리/NNG,하락/NNG,하/VV,만기/NNG...","소비자/NNG;신뢰지수/NNG;예상/NNG;하회/NNG,물가/NNG;상승/NNG;압...","채권/NNG,시황/NNG,만기/NNG,금리/NNG,하락/NNG,하/VV,만기/NNG...",5.03,4.94,1
1319,2008-05-28,2008-04-28,"채권/NNG,시황/NNG,만기/NNG,금리/NNG,상승/NNG,하/VV,만기/NNG...","미/NNG;국채/NNG;수익률/NNG;급락/NNG,낙찰/NNG;금리/NNG;예상/N...","채권/NNG,시황/NNG,만기/NNG,금리/NNG,상승/NNG,하/VV,만기/NNG...",5.03,4.89,1
1320,2008-05-23,2008-04-23,"채권/NNG,시황/NNG,만기/NNG,금리/NNG,상승/NNG,하/VV,만기/NNG...","금리/NNG;하락/NNG;가능성/NNG;낮/VA,당분간/MAG;금리/NNG;하락/N...","채권/NNG,시황/NNG,만기/NNG,금리/NNG,상승/NNG,하/VV,만기/NNG...",5.00,4.98,0
1321,2008-05-22,2008-04-22,"채권/NNG,시황/NNG,만기/NNG,금리/NNG,하락/NNG,하/VV,만기/NNG...","물가/NNG;상승/NNG;압력/NNG,물가/NNG;상승/NNG;견인/NNG,금리/N...","채권/NNG,시황/NNG,만기/NNG,금리/NNG,하락/NNG,하/VV,만기/NNG...",5.00,4.98,0


In [144]:
data_original.to_csv(r'/content/drive/My Drive/자연어 처리 1조/데이터셋/report_original.csv')

In [138]:
data_fin = data[['token_ngram', 'label']]
data_fin

,token_ngram,label
0,"프레젠테이션/NNG,채권/NNG,채권시장/NNG,전망/NNG,동향/NNG,금리/NN...",1
1,"의사록/NNG,확신/NNG,없/VA,인상/NNG,의견/NNG,있/VA,확인/NNG,...",1
2,"속도조절/NNG,중/NNG,발행시장/NNG,공모/NNG,회사채/NNG,발행/NNG,...",1
3,"fed/NNG,fed/NNG,함의/NNG,fed/NNG,구성/NNG,변화/NNG,채...",1
4,"채권분석/NNG,채권/NNG,fed/NNG,통화정책/NNG,불확실성/NNG,확대/N...",1
...,...,...
1318,"채권/NNG,시황/NNG,만기/NNG,금리/NNG,하락/NNG,하/VV,만기/NNG...",1
1319,"채권/NNG,시황/NNG,만기/NNG,금리/NNG,상승/NNG,하/VV,만기/NNG...",1
1320,"채권/NNG,시황/NNG,만기/NNG,금리/NNG,상승/NNG,하/VV,만기/NNG...",0
1321,"채권/NNG,시황/NNG,만기/NNG,금리/NNG,하락/NNG,하/VV,만기/NNG...",0


In [145]:
data_fin.to_csv(r'/content/drive/My Drive/자연어 처리 1조/데이터셋/report_handle.csv')